In [1]:
import torchvision
import torch.nn as nn
import torch.nn.functional as nnf
import torch
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms.functional import to_tensor, normalize
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output
from torchvision import transforms, models
from torchvision.datasets import Food101
from torchvision.models import ResNet50_Weights
import os
import shutil
from torchvision import datasets, transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
print(torch.cuda.get_device_name(0))

def test_acc(net: nn.Module, test_loader: DataLoader):

  net.to(device)
  net.eval()
  
  total = 0
  correct = 0
  top2_correct = 0

  for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    total += labels.size(0)

    outputs = net(images)
    _, predicted = torch.max(outputs, 1)
    correct += (predicted == labels).sum().item()

    _, predicted2 = torch.topk(outputs, 2, dim=1)  # Get top 2 predictions
    top2_correct += ((predicted2 == labels.view(-1, 1)) | (predicted2[:, 1].view(-1, 1) == labels.view(-1, 1))).sum().item()

  return (correct / total * 100, top2_correct / total * 100)

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        resnet = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
        resnet = nn.Sequential(*list(resnet.children())[:-1])
        for param in resnet.parameters():
            param.requires_grad = False


        self.model = nn.Sequential(
            resnet,
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.25),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.1),
            nn.Linear(128, 101)
        )

    def forward(self, x):
        return self.model(x)

cuda
NVIDIA GeForce RTX 4060 Laptop GPU


In [2]:
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_dir = "C:/Users/Vlad Talpiga.VLR_PROJAMZ/OneDrive - Valrom Industrie SRL/Desktop/Facultate/IAVA/Proiect/Food Classifier Final/datasets/google_images_101"

test_dataset = datasets.ImageFolder(test_dir, transform=transform)

batch_size = 32
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
model = Model()

start = time.time()
for i in range(1, 13):

    state_dict = torch.load(f'saved_models_aug/aug_epoch{i}.pkl')
    model.load_state_dict(state_dict)

    accuracy, top2accuracy = test_acc(model, test_loader)

    print(f'Epoch {i}: accuracy - {accuracy:.3f}%, top 2 accuracy - {top2accuracy:.3f}%')

print(f'Testing duration: {time.time() - start}')

Epoch 1: accuracy - 39.382%, top 2 accuracy - 67.053%
Epoch 2: accuracy - 42.302%, top 2 accuracy - 70.698%
Epoch 3: accuracy - 44.296%, top 2 accuracy - 73.155%
Epoch 4: accuracy - 45.172%, top 2 accuracy - 73.024%
Epoch 5: accuracy - 42.644%, top 2 accuracy - 71.664%
Epoch 6: accuracy - 45.454%, top 2 accuracy - 74.212%
Epoch 7: accuracy - 44.638%, top 2 accuracy - 72.148%


KeyboardInterrupt: 

In [8]:
model = Model()

start = time.time()
for i in range(1, 13):

    state_dict = torch.load(f'saved_models_not_aug/model_epoch{i}.pkl')
    model.load_state_dict(state_dict)

    accuracy, top2accuracy = test_acc(model, test_loader)

    print(f'Epoch {i}: accuracy - {accuracy:.3f}%, top 2 accuracy - {top2accuracy:.3f}%')

print(f'Testing duration: {time.time() - start}')

Epoch 1: accuracy - 50.599%, top 2 accuracy - 78.431%
Epoch 2: accuracy - 54.325%, top 2 accuracy - 80.385%
Epoch 3: accuracy - 55.855%, top 2 accuracy - 82.499%
Epoch 4: accuracy - 55.372%, top 2 accuracy - 81.935%
Epoch 5: accuracy - 57.587%, top 2 accuracy - 83.526%
Epoch 6: accuracy - 57.285%, top 2 accuracy - 83.164%
Epoch 7: accuracy - 57.547%, top 2 accuracy - 83.124%
Epoch 8: accuracy - 58.947%, top 2 accuracy - 83.275%
Epoch 9: accuracy - 56.913%, top 2 accuracy - 83.617%
Epoch 10: accuracy - 58.876%, top 2 accuracy - 85.480%
Epoch 11: accuracy - 59.803%, top 2 accuracy - 85.379%
Epoch 12: accuracy - 60.105%, top 2 accuracy - 85.137%
Testing duration: 3452.500349521637


In [9]:
model = Model()

start = time.time()
for i in range(1, 13):

    state_dict = torch.load(f'saved_models_double/epoch{i}.pkl')
    model.load_state_dict(state_dict)

    accuracy, top2accuracy = test_acc(model, test_loader)

    print(f'Epoch {i}: accuracy - {accuracy:.3f}%, top 2 accuracy - {top2accuracy:.3f}%')

print(f'Testing duration: {time.time() - start}')

Epoch 1: accuracy - 58.705%, top 2 accuracy - 83.959%
Epoch 2: accuracy - 59.219%, top 2 accuracy - 84.010%
Epoch 3: accuracy - 60.367%, top 2 accuracy - 86.567%
Epoch 4: accuracy - 60.689%, top 2 accuracy - 85.017%
Epoch 5: accuracy - 60.044%, top 2 accuracy - 86.084%
Epoch 6: accuracy - 60.427%, top 2 accuracy - 86.527%
Epoch 7: accuracy - 60.346%, top 2 accuracy - 85.500%
Epoch 8: accuracy - 58.544%, top 2 accuracy - 84.785%
Epoch 9: accuracy - 60.820%, top 2 accuracy - 85.711%
Epoch 10: accuracy - 59.984%, top 2 accuracy - 84.694%
Epoch 11: accuracy - 59.903%, top 2 accuracy - 84.775%
Epoch 12: accuracy - 60.548%, top 2 accuracy - 84.755%
Testing duration: 5568.075646162033


In [3]:
model = Model()

start = time.time()
for i in range(1, 11):

    state_dict = torch.load(f'saved_models_triple/epoch{i}.pkl')
    model.load_state_dict(state_dict)

    accuracy, top2accuracy = test_acc(model, test_loader)

    print(f'Epoch {i}: accuracy - {accuracy:.3f}%, top 2 accuracy - {top2accuracy:.3f}%')

print(f'Testing duration: {time.time() - start}')

Epoch 1: accuracy - 56.641%, top 2 accuracy - 82.600%
Epoch 2: accuracy - 57.497%, top 2 accuracy - 83.033%
Epoch 3: accuracy - 58.081%, top 2 accuracy - 83.919%
Epoch 4: accuracy - 57.940%, top 2 accuracy - 84.100%
Epoch 5: accuracy - 55.886%, top 2 accuracy - 83.698%
Epoch 6: accuracy - 58.262%, top 2 accuracy - 83.677%
Epoch 7: accuracy - 58.202%, top 2 accuracy - 83.597%
Epoch 8: accuracy - 57.426%, top 2 accuracy - 82.197%
Epoch 9: accuracy - 58.635%, top 2 accuracy - 83.929%
Epoch 10: accuracy - 58.282%, top 2 accuracy - 84.322%
Testing duration: 3002.9676978588104
